In [2]:
# Common imports
import numpy as np
import os
import sys
import gc

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt


import cv2
import numpy as np
import os

import os
import cloudpickle
from multiprocessing import Pool

class RunAsCUDASubprocess:
    def __init__(self, num_gpus=0, memory_fraction=0.8):
        self._num_gpus = num_gpus
        self._memory_fraction = memory_fraction

    @staticmethod
    def _subprocess_code(num_gpus, memory_fraction, fn, args):
        # set the env vars inside the subprocess so that we don't alter the parent env
        os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see tensorflow issue #152
        try:
            import py3nvml
            num_grabbed = py3nvml.grab_gpus(num_gpus, gpu_fraction=memory_fraction)
        except:
            # either CUDA is not installed on the system or py3nvml is not installed (which probably means the env
            # does not have CUDA-enabled packages). Either way, block the visible devices to be sure.
            num_grabbed = 0
            os.environ['CUDA_VISIBLE_DEVICES'] = ""

        assert num_grabbed == num_gpus, 'Could not grab {} GPU devices with {}% memory available'.format(
            num_gpus,
            memory_fraction * 100)
        if os.environ['CUDA_VISIBLE_DEVICES'] == "":
            os.environ['CUDA_VISIBLE_DEVICES'] = "-1"  # see tensorflow issues: #16284, #2175

        # using cloudpickle because it is more flexible about what functions it will
        # pickle (lambda functions, notebook code, etc.)
        return cloudpickle.loads(fn)(*args)

    def __call__(self, f):
        def wrapped_f(*args):
            with Pool(1) as p:
                return p.apply(RunAsCUDASubprocess._subprocess_code, (self._num_gpus, self._memory_fraction, cloudpickle.dumps(f), args))

        return wrapped_f

In [3]:
target_video_path = '/home/john/ml/videos/John Resume Start.mov'
save_dir = '/home/john/ml/videos/Frame_Dump/'

if False:
    '''
    Using OpenCV takes a mp4 video and produces a number of images.
    Requirements
    ----
    You require OpenCV 3.2 to be installed.
    Run
    ----
    Open the main.py and edit the path to the video. Then run:
    $ python main.py
    Which will produce a folder called data with the images. There will be 2000+ images for example.mp4.
    '''
    # Playing video from file:
    cap = cv2.VideoCapture(target_video_path)

    try:
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
    except OSError:
        print ('Error: Creating directory of data')

    currentFrame = 0
    while(True):
        # Capture frame-by-frame
        ret, frame = cap.read()

        # Saves image of the current frame in jpg file
        name = save_dir + str(currentFrame) + '.png'
        print ('Creating...' + name)
        cv2.imwrite(name, frame)

        # To stop duplicate images
        currentFrame += 1

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()

In [4]:
from keras.preprocessing.image import load_img, img_to_array

target_image_path = '../ml/elephants.jpeg'
style_reference_image_path = '../ml/Dream Images/starry night.jpg'

width, height = load_img('../ml/elephants.jpeg').size
img_height = 400
img_width = int(width * img_height / height)

In [5]:
import numpy as np
from keras.applications import vgg19

#@RunAsCUDASubprocess(num_gpus=1, memory_fraction=0.8)
def produce_frame(iterations_per_frame):
    def preprocess_image(image_path):
        img = load_img(image_path, target_size=(img_height, img_width))
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = vgg19.preprocess_input(img)
        return img

    def deprocess_image(x):
        x[:, :, 0] += 103.939
        x[:, :, 1] += 116.779
        x[:, :, 2] += 123.68
        x = x[:, :, ::-1]
        x = np.clip(x, 0, 255).astype('uint8')
        return x
    def content_loss(base, combination):
        return K.sum(K.square(combination - base))

    def gram_matrix(x):
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
        gram = K.dot(features, K.transpose(features))
        return gram

    def style_loss(style, combination):
        S = gram_matrix(style)
        C = gram_matrix(combination)
        channels = 3
        size = img_height * img_width
        return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size **2))

    def total_variation_loss(x):
        a = K.square(
            x[:, :img_height - 1, :img_width - 1, :] -
            x[:, 1:, :img_width - 1, :])
        b = K.square(
            x[:, :img_height - 1, :img_width - 1, :] -
            x[:, :img_height - 1, 1:, :])
        return K.sum(K.pow(a + b, 1.25))

    class Evaluator(object):

        def __init__(self):
            self.loss_value = None
            self.grads_values = None

        def loss(self, x):
            assert self.loss_value is None
            x = x.reshape((1, img_height, img_width, 3))
            outs = fetch_loss_and_grads([x])
            loss_value = outs[0]
            grad_values = outs[1].flatten().astype('float64')
            self.loss_value = loss_value
            self.grad_values = grad_values
            return self.loss_value

        def grads(self, x):
            assert self.loss_value is not None
            grad_values = np.copy(self.grad_values)
            self.loss_value = None
            self.grad_values = None
            return grad_values
    from keras import backend as K
    from scipy.optimize import fmin_l_bfgs_b
    from scipy.misc import imsave
    import time
    import gc




    target_image = K.constant(preprocess_image(target_image_path))
    style_reference_image = K.constant(preprocess_image(style_reference_image_path))
    combination_image = K.placeholder((1, img_height, img_width, 3))

    input_tensor = K.concatenate([target_image,
                                  style_reference_image,
                                  combination_image], axis=0)

    model = vgg19.VGG19(input_tensor=input_tensor,
                        weights='imagenet',
                        include_top=False)

    print('Model loaded.')


    outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
    content_layer = 'block5_conv2'
    style_layers = ['block1_conv1',
                    'block2_conv1',
                    'block3_conv1',
                    'block4_conv1',
                    'block5_conv1']
    total_variation_weight = 1e-4
    style_weight = 1.
    content_weight = 0.025
    loss = K.variable(0.)
    layer_features = outputs_dict[content_layer]
    target_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss += content_weight * content_loss(target_image_features, combination_features)

    for layer_name in style_layers:
        layer_features = outputs_dict[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        sl = style_loss(style_reference_features, combination_features)
        loss += (style_weight / len(style_layers)) * sl

    loss += total_variation_weight * total_variation_loss(combination_image)

    grads = K.gradients(loss, combination_image)[0]

    fetch_loss_and_grads = K.function([combination_image], [loss, grads])

    evaluator = Evaluator()




    result_prefix = 'starry'
    x = preprocess_image(target_image_path)
    x = x.flatten()
    for iteration in range(iterations_per_frame):
        print('Start of iteration', iteration)
        start_time = time.time()
        x, min_val, info = fmin_l_bfgs_b(evaluator.loss,
                                         x,
                                         fprime=evaluator.grads,
                                         maxfun=20)
        print('Current loss value:', min_val)
        img = x.copy().reshape((img_height, img_width, 3))
        img = deprocess_image(img)
        fname = result_prefix + '_at_iteration_%d.png' % iteration
        imsave('../ml/result/' + fname, img)
        print('Image saved as', fname)
        end_time = time.time()
        print('Iteration %d completed in %ds' % (iteration, end_time - start_time))

    del model
    gc.collect()

iterations = 20
produce_frame(iterations)

Model loaded.
Start of iteration 0
Current loss value: 1026368300.0
Image saved as starry_at_iteration_0.png
Iteration 0 completed in 24s
Start of iteration 1


/home/john/ml/env/lib/python3.6/site-packages/ipykernel_launcher.py:138: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Current loss value: 372517950.0
Image saved as starry_at_iteration_1.png
Iteration 1 completed in 18s
Start of iteration 2
Current loss value: 232667440.0
Image saved as starry_at_iteration_2.png
Iteration 2 completed in 18s
Start of iteration 3
Current loss value: 177029250.0
Image saved as starry_at_iteration_3.png
Iteration 3 completed in 18s
Start of iteration 4
Current loss value: 148568290.0
Image saved as starry_at_iteration_4.png
Iteration 4 completed in 18s
Start of iteration 5
Current loss value: 132701800.0
Image saved as starry_at_iteration_5.png
Iteration 5 completed in 18s
Start of iteration 6
Current loss value: 120935390.0
Image saved as starry_at_iteration_6.png
Iteration 6 completed in 18s
Start of iteration 7
Current loss value: 111708376.0
Image saved as starry_at_iteration_7.png
Iteration 7 completed in 18s
Start of iteration 8
Current loss value: 104472310.0
Image saved as starry_at_iteration_8.png
Iteration 8 completed in 18s
Start of iteration 9
Current loss val